In [88]:
import numpy as np
import pandas as pd
from google.colab import files
from google.colab import drive
import glob
import os
import matplotlib.pyplot as plt
import scipy.special as spsp
import scipy.stats as spst
from pandas.tseries.offsets import MonthEnd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
COV_DIR = "/content/drive/MyDrive/DDDM_Project/Data/Covariates"

TB3MS     = pd.read_csv(os.path.join(COV_DIR, "TB3MS.csv"))
DGS10     = pd.read_csv(os.path.join(COV_DIR, "DGS10.csv"))
BAA       = pd.read_csv(os.path.join(COV_DIR, "BAA.csv"))
VIXCLS    = pd.read_csv(os.path.join(COV_DIR, "VIXCLS.csv"))
UNRATE    = pd.read_csv(os.path.join(COV_DIR, "UNRATE.csv"))
CPIAUCSL  = pd.read_csv(os.path.join(COV_DIR, "CPIAUCSL.csv"))
ETF_IR    = pd.read_csv(os.path.join(COV_DIR, "ETF+IR.csv"))

In [90]:
# making all dates datetime data type

for df in [TB3MS, DGS10, BAA, VIXCLS, UNRATE, CPIAUCSL]:
    for col in ["DATE", "observation_date"]:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])

In [91]:
ETF_IR = ETF_IR.sort_values("Month").reset_index(drop=True)
ETF_IR = ETF_IR.rename(columns={
    "Pct_Growth": "Growth",
    "rf_month_pct": "rf_month"
})

# Convert rf_month from percent → decimal
ETF_IR["rf_month"] = ETF_IR["rf_month"] / 100.0

ETF_IR.head()

,Month,Open,Close,Growth,ETF,rf_month
0,1993-01-31,43.96875,43.93750,-0.000711,SPY,0.002466
1,1993-02-28,43.96875,44.40625,0.009950,SPY,0.002409
2,1993-03-31,44.56250,45.18750,0.014025,SPY,0.002426
3,1993-04-30,45.25000,44.03125,-0.026934,SPY,0.002361
4,1993-05-31,44.09375,45.21875,0.025514,SPY,0.002434


In [92]:
# build binary outperformed output variable and continous excess return variable

ETF_IR["outperformed"] = (ETF_IR["Growth"] > ETF_IR["rf_month"]).astype(int)
ETF_IR["excess_return"] = ETF_IR["Growth"] - ETF_IR["rf_month"]

ETF_IR.head()

,Month,Open,Close,Growth,ETF,rf_month,outperformed,excess_return
0,1993-01-31,43.96875,43.93750,-0.000711,SPY,0.002466,0,-0.003177
1,1993-02-28,43.96875,44.40625,0.009950,SPY,0.002409,1,0.007541
2,1993-03-31,44.56250,45.18750,0.014025,SPY,0.002426,1,0.011600
3,1993-04-30,45.25000,44.03125,-0.026934,SPY,0.002361,0,-0.029294
4,1993-05-31,44.09375,45.21875,0.025514,SPY,0.002434,1,0.023080


In [93]:
# check outperformance rate

print(ETF_IR["outperformed"].mean())
print(ETF_IR["excess_return"].mean())

0.5692751590982634
0.0035306609551990094


In [94]:
# build integer context outperform_streak variable

# 1) Sort so streaks are computed in time order within each ETF
ETF_IR = ETF_IR.sort_values(["ETF", "Month"]).reset_index(drop=True)

# 2) Prior-month consecutive outperformance streak (leak-free)
def prior_streak(y):
    out = np.zeros(len(y), dtype=int)
    streak = 0
    for i, v in enumerate(y):
        out[i] = streak          # store PRIOR streak
        streak = streak + 1 if v == 1 else 0
    return out

ETF_IR["months_outperforming"] = (
    ETF_IR.groupby("ETF")["outperformed"].transform(prior_streak)
)

ETF_IR.head()

,Month,Open,Close,Growth,ETF,rf_month,outperformed,excess_return,months_outperforming
0,2015-03-31,17.719999,19.420000,0.095937,AFTY,0.000025,1,0.095912,0
1,2015-04-30,19.910000,22.820000,0.146158,AFTY,0.000017,1,0.146141,1
2,2015-05-31,23.020000,21.870001,-0.049957,AFTY,0.000017,0,-0.049973,2
3,2015-06-30,22.820000,20.730000,-0.091586,AFTY,0.000017,0,-0.091603,0
4,2015-07-31,19.980000,18.250000,-0.086587,AFTY,0.000025,0,-0.086612,0


In [95]:
TB3MS.head()

,observation_date,TB3MS
0,1934-01-01,0.72
1,1934-02-01,0.62
2,1934-03-01,0.24
3,1934-04-01,0.15
4,1934-05-01,0.16


In [96]:
# make date month_end of same month

TB3MS["DATE"] = pd.to_datetime(TB3MS["observation_date"])
TB3MS["Month"] = TB3MS["DATE"] + MonthEnd(0)

TB3MS = TB3MS.groupby("Month", as_index=False).mean(numeric_only=True)

TB3MS.head()

,Month,TB3MS
0,1934-01-31,0.72
1,1934-02-28,0.62
2,1934-03-31,0.24
3,1934-04-30,0.15
4,1934-05-31,0.16


In [97]:
# building change in IR

TB3MS = TB3MS.sort_values("Month").reset_index(drop=True)

TB3MS_change = TB3MS.copy()
TB3MS_change["dTB3MS"] = TB3MS_change["TB3MS"].diff()

TB3MS_change = TB3MS_change[["Month", "dTB3MS"]]

TB3MS_change.head()

,Month,dTB3MS
0,1934-01-31,NaN
1,1934-02-28,-0.10
2,1934-03-31,-0.38
3,1934-04-30,-0.09
4,1934-05-31,0.01


In [98]:
DGS10.head()

,observation_date,DGS10
0,1962-01-02,4.06
1,1962-01-03,4.03
2,1962-01-04,3.99
3,1962-01-05,4.02
4,1962-01-08,4.03


In [99]:
# averaging DGS10 rate by month

DGS10["observation_date"] = pd.to_datetime(DGS10["observation_date"])
DGS10["Month"] = DGS10["observation_date"] + MonthEnd(0)

DGS10_m = (
    DGS10
    .groupby("Month", as_index=False)
    .mean(numeric_only=True)
)

DGS10_m.head()

,Month,DGS10
0,1962-01-31,4.083182
1,1962-02-28,4.039444
2,1962-03-31,3.930455
3,1962-04-30,3.843000
4,1962-05-31,3.873636


In [100]:
# building Yield Curve Slope

DGS10_m = DGS10_m.sort_values("Month").reset_index(drop=True)
TB3MS_m = TB3MS.sort_values("Month").reset_index(drop=True)

# Merge on Month
yield_curve = pd.merge(
    DGS10_m,
    TB3MS_m,
    on="Month",
    how="inner"
)

# Compute slope
yield_curve["yield_slope"] = yield_curve["DGS10"] - yield_curve["TB3MS"]

yield_curve = yield_curve[["Month", "yield_slope"]]

yield_curve.head()

,Month,yield_slope
0,1962-01-31,1.363182
1,1962-02-28,1.309444
2,1962-03-31,1.210455
3,1962-04-30,1.113000
4,1962-05-31,1.183636


In [101]:
BAA.head()

,observation_date,BAA
0,1919-01-01,7.12
1,1919-02-01,7.20
2,1919-03-01,7.15
3,1919-04-01,7.23
4,1919-05-01,7.09


In [102]:
# making date month_end of same month

BAA["observation_date"] = pd.to_datetime(BAA["observation_date"])
BAA["Month"] = BAA["observation_date"] + MonthEnd(0)

BAA_m = BAA.groupby("Month", as_index=False).mean(numeric_only=True)

BAA_m.head()

,Month,BAA
0,1919-01-31,7.12
1,1919-02-28,7.20
2,1919-03-31,7.15
3,1919-04-30,7.23
4,1919-05-31,7.09


In [103]:
# building credit spread

BAA_m = BAA_m.sort_values("Month").reset_index(drop=True)
DGS10_m = DGS10_m.sort_values("Month").reset_index(drop=True)

# Merge on Month
credit_spread = pd.merge(
    BAA_m,
    DGS10_m,
    on="Month",
    how="inner"
)

# Compute credit spread
credit_spread["credit_spread"] = credit_spread["BAA"] - credit_spread["DGS10"]

credit_spread = credit_spread[["Month", "credit_spread"]]

credit_spread.head()

,Month,credit_spread
0,1962-01-31,0.996818
1,1962-02-28,1.030556
2,1962-03-31,1.109545
3,1962-04-30,1.177000
4,1962-05-31,1.126364


In [104]:
VIXCLS.head()

,observation_date,VIXCLS
0,1990-01-02,17.24
1,1990-01-03,18.19
2,1990-01-04,19.22
3,1990-01-05,20.11
4,1990-01-08,20.26


In [105]:
# averaging VIX by month

VIXCLS["observation_date"] = pd.to_datetime(VIXCLS["observation_date"])

# Snap each day to its month-end
VIXCLS["Month"] = VIXCLS["observation_date"] + MonthEnd(0)

# Monthly average
VIX_m = (
    VIXCLS
    .groupby("Month", as_index=False)
    .mean(numeric_only=True)
)

VIX_m.head()

,Month,VIXCLS
0,1990-01-31,23.347273
1,1990-02-28,23.262632
2,1990-03-31,20.062273
3,1990-04-30,21.403500
4,1990-05-31,18.097727


In [106]:
UNRATE.head()

,observation_date,UNRATE
0,1948-01-01,3.4
1,1948-02-01,3.8
2,1948-03-01,4.0
3,1948-04-01,3.9
4,1948-05-01,3.5


In [107]:
# making date month_end of same month

UNRATE["observation_date"] = pd.to_datetime(UNRATE["observation_date"])
UNRATE["Month"] = UNRATE["observation_date"] + MonthEnd(0)

UNRATE_m = UNRATE.groupby("Month", as_index=False).mean(numeric_only=True)

UNRATE_m.head()

,Month,UNRATE
0,1948-01-31,3.4
1,1948-02-29,3.8
2,1948-03-31,4.0
3,1948-04-30,3.9
4,1948-05-31,3.5


In [108]:
CPIAUCSL.head()

,observation_date,CPIAUCSL
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


In [109]:
# make date month_end of same month

CPIAUCSL["observation_date"] = pd.to_datetime(CPIAUCSL["observation_date"])
CPIAUCSL["Month"] = CPIAUCSL["observation_date"] + MonthEnd(0)

CPIAUCSL_m = CPIAUCSL.groupby("Month", as_index=False).mean(numeric_only=True)

CPIAUCSL_m.head()

,Month,CPIAUCSL
0,1947-01-31,21.48
1,1947-02-28,21.62
2,1947-03-31,22.00
3,1947-04-30,22.00
4,1947-05-31,21.95


In [110]:
# building yearly inflation and monthly inflation shock

CPIAUCSL_m = CPIAUCSL_m.sort_values("Month").reset_index(drop=True)

inflation = CPIAUCSL_m[["Month", "CPIAUCSL"]].copy()

# 1) Year-over-year inflation
#    Decimal form: 0.02 = 2%
inflation["inflation_yoy"] = inflation["CPIAUCSL"].pct_change(12)


# 2) Monthly inflation (intermediate)
inflation["inflation_mom"] = inflation["CPIAUCSL"].pct_change(1)

# 3) Inflation shock (SECONDARY)
#    De-meaned monthly inflation
inflation["inflation_shock"] = (
    inflation["inflation_mom"]
    - inflation["inflation_mom"].rolling(12).mean()
)


inflation = inflation[["Month", "inflation_yoy", "inflation_shock"]]

inflation.head(20)

,Month,inflation_yoy,inflation_shock
0,1947-01-31,NaN,NaN
1,1947-02-28,NaN,NaN
2,1947-03-31,NaN,NaN
3,1947-04-30,NaN,NaN
4,1947-05-31,NaN,NaN
5,1947-06-30,NaN,NaN
6,1947-07-31,NaN,NaN
7,1947-08-31,NaN,NaN
8,1947-09-30,NaN,NaN
9,1947-10-31,NaN,NaN


In [111]:

# -----------------------------
# 0) Helper: keep only needed cols, enforce Month datetime, de-dup
# -----------------------------
def prep_monthly(df, cols):
    out = df[cols].copy()
    out["Month"] = pd.to_datetime(out["Month"])
    out = out.drop_duplicates(subset=["Month"]).sort_values("Month").reset_index(drop=True)
    return out

TB3MS_level   = prep_monthly(TB3MS,        ["Month", "TB3MS"])  # annualized % level
TB3MS_change_ = prep_monthly(TB3MS_change, ["Month", "dTB3MS"])
UNRATE_       = prep_monthly(UNRATE_m,     ["Month", "UNRATE"])
inflation_    = prep_monthly(inflation,    ["Month", "inflation_yoy", "inflation_shock"])
yc_           = prep_monthly(yield_curve,  ["Month", "yield_slope"])
cs_           = prep_monthly(credit_spread,["Month", "credit_spread"])
vix_          = prep_monthly(VIX_m,        ["Month", "VIXCLS"])

# -----------------------------
# 1) Merge monthly macro tables (inner join to keep aligned months)
# -----------------------------
macro = TB3MS_level
for df in [TB3MS_change_, UNRATE_, inflation_, yc_, cs_, vix_]:
    macro = pd.merge(macro, df, on="Month", how="inner")

macro = macro.sort_values("Month").reset_index(drop=True)

# -----------------------------
# 2) Lag variables
# -----------------------------
macro_cols = [
    "TB3MS", "dTB3MS", "UNRATE",
    "inflation_yoy", "inflation_shock",
    "yield_slope", "credit_spread", "VIXCLS"
]

for col in macro_cols:
    macro[col + "_lag1"] = macro[col].shift(1)

macro_lagged = macro[["Month"] + [c + "_lag1" for c in macro_cols]]

# -----------------------------
# 3) Merge
# -----------------------------
ETF_IR["Month"] = pd.to_datetime(ETF_IR["Month"])
ETF_IR = ETF_IR.sort_values(["ETF", "Month"]).reset_index(drop=True)

ETF_IR = ETF_IR.merge(
    macro_lagged,
    on="Month",
    how="left"
)

# drop contemporaneous macro columns (keep only lagged)
#ETF_IR = ETF_IR.drop(columns=macro_cols)

# -----------------------------
# 4) Quick checks
# -----------------------------
print("macro shape:", macro.shape)
print("ETF_IR shape:", ETF_IR.shape)
print("macro date range:", macro["Month"].min(), "→", macro["Month"].max())

macro shape: (429, 17)
ETF_IR shape: (74168, 17)
macro date range: 1990-01-31 00:00:00 → 2025-09-30 00:00:00


In [112]:
ETF_IR.head()

,Month,Open,Close,Growth,ETF,rf_month,outperformed,excess_return,months_outperforming,TB3MS_lag1,dTB3MS_lag1,UNRATE_lag1,inflation_yoy_lag1,inflation_shock_lag1,yield_slope_lag1,credit_spread_lag1,VIXCLS_lag1
0,2015-03-31,17.719999,19.420000,0.095937,AFTY,0.000025,1,0.095912,0,0.02,-0.01,5.5,-0.000870,0.002604,1.955263,2.534737,15.904211
1,2015-04-30,19.910000,22.820000,0.146158,AFTY,0.000017,1,0.146141,1,0.03,0.01,5.4,-0.000220,0.002709,2.012727,2.497273,14.812727
2,2015-05-31,23.020000,21.870001,-0.049957,AFTY,0.000017,0,-0.049973,2,0.02,-0.01,5.4,-0.001040,0.001126,1.915000,2.545000,13.494762
3,2015-06-30,22.820000,20.730000,-0.091586,AFTY,0.000017,0,-0.091603,0,0.02,0.00,5.6,0.000350,0.003265,2.177500,2.692500,13.338500
4,2015-07-31,19.980000,18.250000,-0.086587,AFTY,0.000025,0,-0.086612,0,0.02,0.00,5.3,0.001796,0.002615,2.343636,2.766364,14.339545


In [114]:
ETF_IR.to_csv(os.path.join(COV_DIR, "ETF_Covars.csv"))

In [120]:
tagged_final=pd.read_csv(os.path.join(COV_DIR, "tagged_final.csv"))
tagged_final.head()

,ETF,is_equity,is_international,is_gov,is_credit,is_macro,needs_review
0,AFTY,1,1,0,0,0,1
1,AGZ,0,0,1,0,0,1
2,AIRR,1,0,0,0,0,0
3,ANGL,0,0,0,1,0,0
4,AOA,1,1,1,1,0,1


In [122]:
final_dataset = ETF_IR.merge(
    tagged_final,
    on="ETF",
    how="left",
    validate="many_to_one"   # each ETF maps to exactly one tag row
)

final_dataset.drop(columns=["needs_review"], inplace=True)

final_dataset.head()

,Month,Open,Close,Growth,ETF,rf_month,outperformed,excess_return,months_outperforming,TB3MS_lag1,...,inflation_yoy_lag1,inflation_shock_lag1,yield_slope_lag1,credit_spread_lag1,VIXCLS_lag1,is_equity,is_international,is_gov,is_credit,is_macro
0,2015-03-31,17.719999,19.420000,0.095937,AFTY,0.000025,1,0.095912,0,0.02,...,-0.000870,0.002604,1.955263,2.534737,15.904211,1,1,0,0,0
1,2015-04-30,19.910000,22.820000,0.146158,AFTY,0.000017,1,0.146141,1,0.03,...,-0.000220,0.002709,2.012727,2.497273,14.812727,1,1,0,0,0
2,2015-05-31,23.020000,21.870001,-0.049957,AFTY,0.000017,0,-0.049973,2,0.02,...,-0.001040,0.001126,1.915000,2.545000,13.494762,1,1,0,0,0
3,2015-06-30,22.820000,20.730000,-0.091586,AFTY,0.000017,0,-0.091603,0,0.02,...,0.000350,0.003265,2.177500,2.692500,13.338500,1,1,0,0,0
4,2015-07-31,19.980000,18.250000,-0.086587,AFTY,0.000025,0,-0.086612,0,0.02,...,0.001796,0.002615,2.343636,2.766364,14.339545,1,1,0,0,0


In [123]:
n_nan_rows = ETF_IR.isna().any(axis=1).sum()
n_nan_cells = ETF_IR.isna().sum().sum()

print("Rows with at least one NaN:", n_nan_rows)
print("Total NaN cells:", n_nan_cells)

Rows with at least one NaN: 0
Total NaN cells: 0


In [125]:
final_dataset.to_csv(os.path.join(COV_DIR, "final_dataset.csv"))